<a href="https://colab.research.google.com/github/andres-stack/Stock-Price-Perdiction/blob/main/svmStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
import yfinance as yf
import datetime
from finta import TA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
numDays = 14000
dataInterval = '1d'
symbol = 'TSLA'
INDICATORS = INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']


start = (datetime.date.today() - datetime.timedelta(numDays))
end = datetime.datetime.today()

data = yf.download(symbol, start=start, end=end, interval=dataInterval)
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}, inplace=True)

print(data)

In [16]:
#Feature Engineering Technical Indicators
for indicator in INDICATORS:
  indicatorValues = eval('TA.' + indicator + '(data)')
  if not isinstance(indicatorValues, pd.DataFrame):
    indicatorValues = indicatorValues.to_frame()
  data = data.merge(indicatorValues, left_index=True, right_index=True)
del (data['open'])
del (data['high'])
del (data['low'])
del (data['volume'])
del (data['Adj Close'])

In [17]:
#finding the truth values
def getprediction(data, window):
    
    prediction = (data.shift(-window)['close'] >= data['close'])
    prediction = prediction.iloc[:-window]
    data['pred'] = prediction.astype(int)
    
    return data

data = getprediction(data, window=15)
del (data['close'])
data = data.dropna() 
data.tail()

,14 period RSI,MACD,SIGNAL,14 period STOCH %K,MFV,14 period ATR,MOM,14 period MFI,ROC,OBV,20 period CCI,14 period EMV.,VIm,VIp,pred
Date,,,,,,,,,,,,,,,
2021-11-01,94.198009,92.277528,61.389009,99.712841,3.714199e+09,44.946424,338.479980,97.181539,47.691607,5.620882e+09,179.172877,35.462633,0.389623,1.380485,0.0
2021-11-02,85.002278,98.663654,68.843938,90.476792,3.706968e+09,48.724993,307.729980,86.703876,39.022331,5.578144e+09,145.283385,36.532353,0.443275,1.377644,0.0
2021-11-03,86.612434,105.881914,76.251533,99.610719,3.739909e+09,52.553567,348.059998,87.536496,39.506500,5.612772e+09,131.864432,37.151438,0.484254,1.351927,0.0
2021-11-04,87.180722,111.610961,83.323419,96.535902,3.739266e+09,52.892138,335.910034,87.911118,42.306224,5.638170e+09,129.999225,39.331090,0.480493,1.425664,0.0
2021-11-05,85.281224,114.203788,89.499492,94.484387,3.736768e+09,52.866425,312.409973,82.973712,41.151534,5.616590e+09,112.603508,36.518465,0.517517,1.385770,0.0


In [18]:
#implementing SVM
X = data.drop('pred', axis=1)
y = data['pred']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [20]:
#feature Scaling
from sklearn import preprocessing
import numpy as np
scaler = preprocessing.StandardScaler().fit(X_train)
scaler = preprocessing.StandardScaler().fit(X_test)
X_testScaled = scaler.transform(X_test)
X_scaled = scaler.transform(X_train)

In [10]:
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_scaled, y_train)

SVC()

In [21]:
y_pred = svclassifier.predict(X_testScaled)

In [24]:
from sklearn import metrics
print("Gaussian Kernel Accuracy:",metrics.accuracy_score(y_test, y_pred))

Gaussian Kernel Accuracy: 0.6232394366197183


In [26]:
#getting accuracy for Different plynomial terms
i = 2
while i < 20:
  polyclassifier = SVC(kernel = 'poly', degree = i)
  polyclassifier.fit(X_scaled, y_train)
  prediction = polyclassifier.predict(X_testScaled)
  print('polynomial degree '+str(i) +':',metrics.accuracy_score(y_test, prediction))
  i = i+1

polynomial degree 2: 0.5880281690140845
polynomial degree 3: 0.6003521126760564
polynomial degree 4: 0.5950704225352113
polynomial degree 5: 0.6003521126760564
polynomial degree 6: 0.5933098591549296
polynomial degree 7: 0.6038732394366197
polynomial degree 8: 0.5897887323943662
polynomial degree 9: 0.5950704225352113
polynomial degree 10: 0.5950704225352113
polynomial degree 11: 0.6003521126760564
polynomial degree 12: 0.5933098591549296
polynomial degree 13: 0.596830985915493
polynomial degree 14: 0.5880281690140845
polynomial degree 15: 0.596830985915493
polynomial degree 16: 0.5845070422535211
polynomial degree 17: 0.5933098591549296
polynomial degree 18: 0.5862676056338029
polynomial degree 19: 0.5915492957746479
